## 8-9. 프로젝트 - Movielens 영화 추천 실습

### 1) 데이터 준비와 전처리   

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding = 'unicode_escape')
movie_to_title = {k:v for k,v in zip(movies['movie_id'], movies['title'])}
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


#### 데이터를 읽어 오려고 위의 코드를 실행하니 'utf-8' codec can't decode byte 0xe9 in position 3114: invalid continuation byte 에러가 발생했습니다. 검색해보니 파일에 utf-8 컴파일 할 수없는 내용이 있다고 해서 파일을 열어보니 중국어로 된 파일임을 확인할수 있었습니다. encoding = 'utf-8'을 입력해도 에러가 나서 더 찾아보고encoding = 'unicode_escape' 로 해결할수 있었습니다. 다행히 데이터를 잘 읽어오네요.

In [6]:
movie_to_title

{1: 'Toy Story (1995)',
 2: 'Jumanji (1995)',
 3: 'Grumpier Old Men (1995)',
 4: 'Waiting to Exhale (1995)',
 5: 'Father of the Bride Part II (1995)',
 6: 'Heat (1995)',
 7: 'Sabrina (1995)',
 8: 'Tom and Huck (1995)',
 9: 'Sudden Death (1995)',
 10: 'GoldenEye (1995)',
 11: 'American President, The (1995)',
 12: 'Dracula: Dead and Loving It (1995)',
 13: 'Balto (1995)',
 14: 'Nixon (1995)',
 15: 'Cutthroat Island (1995)',
 16: 'Casino (1995)',
 17: 'Sense and Sensibility (1995)',
 18: 'Four Rooms (1995)',
 19: 'Ace Ventura: When Nature Calls (1995)',
 20: 'Money Train (1995)',
 21: 'Get Shorty (1995)',
 22: 'Copycat (1995)',
 23: 'Assassins (1995)',
 24: 'Powder (1995)',
 25: 'Leaving Las Vegas (1995)',
 26: 'Othello (1995)',
 27: 'Now and Then (1995)',
 28: 'Persuasion (1995)',
 29: 'City of Lost Children, The (1995)',
 30: 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 31: 'Dangerous Minds (1995)',
 32: 'Twelve Monkeys (1995)',
 33: 'Wings of Courage (1995)',
 34: 'Babe (1

#### 영화 제목들을 살펴보니 예전에 보았던 영화 제목들이 많이보여 반갑네요. 그때가 좋았는데 ... ㅎㅎㅎ

### 2) 분석해 봅시다.   

In [7]:
# 영화개수
ratings['movie_id'].nunique()

3628

In [8]:
# 사용자수
ratings['user_id'].nunique()

6039

In [9]:
# 가장 인기 있는 영화 30개
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [10]:
# 유저별 몇 편의 영화를 보고 있는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [11]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = ratings.groupby('user_id')['count'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: count, dtype: float64

#### 추천 모델을 만들기 위해서 위와 같이 데이터의 기본 정보들을 확인해 보았습니다.

### 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [12]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
# 356-Forrest Gump, 597-Pretty Woman, 270-Love Affair, 1097-E.T.,318-Shawshank Redemption
my_favorite = ['356' , '597' ,'270' ,'1097' ,'318']

# 'babmu'이라는 user_id가 위 아티스트의 노래를 2회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['babmu']*5, 'movie_id': my_favorite, 'count':[2]*5})

if not ratings.isin({'user_id':['babmu']})['user_id'].any():  # user_id에 'babmu'이라는 데이터가 없다면
    ratings = ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,babmu,356,2,NaN
1,babmu,597,2,NaN
2,babmu,270,2,NaN
3,babmu,1097,2,NaN
4,babmu,318,2,NaN


#### 나의 취향과 가장 유사한 영화를 추천받기 위해서  내가 좋아하는 영화 정보 5개를 추가 해 주었습니다.  'babmu'라는 아이디와 영화 아이디가 잘 추가 되었군요.

In [13]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['babmu'])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039가 나와야 합니다. 
print(movie_to_idx['356'])

6039
3628


In [14]:
# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # ratings['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie_id 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie_id column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

ratings

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,count,timestamp
0,0,0,5,978300760.0
1,0,1,3,978302109.0
2,0,2,3,978301968.0
3,0,3,4,978300275.0
4,0,4,5,978824291.0
...,...,...,...,...
0,6039,3628,2,NaN
1,6039,3629,2,NaN
2,6039,3630,2,NaN
3,6039,3631,2,NaN


#### ratings의 user_id와 movie_id 컬럼 내 값들이 모두 정수 인덱스 값으로 잘 변경되었습니다. 이것으로 훈련을 위한 전처리가 완료 되었네요.

### 4) CSR matrix를 직접 만들어 봅시다.

In [15]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3633 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

#### Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보만으로 구성하여 메모리 사용량을 최소화하면서도 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 CSR matrix를 구성하였습니다.

### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [16]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [17]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [18]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3633x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [19]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [20]:
babmu, fg = user_to_idx['babmu'], movie_to_idx[356]
babmu_vector, fg_vector = als_model.user_factors[babmu], als_model.item_factors[fg]

print('슝=3')

슝=3


In [21]:
babmu_vector

array([-0.00298638,  0.00359355,  0.00584644,  0.00749107, -0.0013956 ,
       -0.00227934, -0.0078551 ,  0.00053512,  0.00442862,  0.00110222,
        0.00833961, -0.00206945,  0.00440345,  0.01272092, -0.0017615 ,
        0.00299444, -0.00545185, -0.0026593 ,  0.003559  ,  0.00026216,
       -0.00768174, -0.0023935 , -0.00340516,  0.00266462,  0.00152411,
       -0.00527054, -0.00274397, -0.00194022,  0.00189212,  0.0039241 ,
       -0.00040673, -0.00046351,  0.00679437,  0.00938676,  0.00226129,
       -0.00429073,  0.00209517, -0.00028705,  0.01075967,  0.00437051,
       -0.00165202, -0.00148669, -0.00471355, -0.00255075, -0.00066331,
       -0.00034344, -0.00116802, -0.00928317, -0.00320181, -0.0092768 ,
        0.00738635,  0.00453165, -0.00756994, -0.00280544, -0.00183371,
        0.01015545, -0.00299   ,  0.00405606,  0.00135176,  0.003452  ,
       -0.00460573, -0.00219135, -0.00376741,  0.00453648,  0.00340453,
       -0.00267395, -0.00679482,  0.00190256, -0.00425623,  0.00

In [22]:
fg_vector

array([-0.00783345, -0.00979702,  0.02454738,  0.0245842 , -0.00416234,
       -0.00722235,  0.01091244,  0.0278669 , -0.00235739,  0.01825741,
       -0.03712902,  0.02080027,  0.03374543, -0.00433935, -0.00272809,
       -0.01043495, -0.02931192, -0.04989834,  0.02544216,  0.02662024,
       -0.00092371, -0.01393222,  0.01632856, -0.01979797,  0.01679974,
        0.00707572,  0.03739345,  0.01338894,  0.02216653,  0.04082609,
       -0.01302994, -0.01260806,  0.00424344, -0.00974842, -0.00013492,
        0.01571656,  0.05350966, -0.01377939,  0.00361261,  0.00748683,
       -0.0046495 ,  0.02001474, -0.0148682 , -0.00308833, -0.00082696,
        0.03321072, -0.02148737,  0.00627894,  0.025078  , -0.00864825,
        0.0424193 , -0.00922756, -0.00085852,  0.03065964,  0.00014605,
       -0.0217982 , -0.0272071 , -0.02198725,  0.0016779 ,  0.02494577,
        0.01501855,  0.01921985,  0.00713954, -0.04016515,  0.02327123,
       -0.00141438,  0.02110872,  0.01080805, -0.0228    , -0.00

In [23]:
# babmu와 fg를 내적하는 코드
np.dot(babmu_vector, fg_vector)

-0.00029153746

#### 생각보다 낮은 수치가 나오네요^^;; 이 모델 잘 학습되었을까요? 제 취향을 잘 고려해 줄지 미지수 입니다. 

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [24]:
Pretty = movie_to_idx[597]
Pretty_vector = als_model.item_factors[Pretty]
np.dot(babmu_vector, Pretty_vector)

0.00013338003

#### 내가 좋아하는 영화중 한편인 American Beauty의 예측 선호도 입니다.

In [25]:
Casino = movie_to_idx[16]
Casino_vector = als_model.item_factors[Casino]
np.dot(babmu_vector, Casino_vector)

-0.00088835007

#### 내가 선호하지 않는 Casino라는 영화의 예측 선호도 입니다. 

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [34]:
favorite_movie = '270'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(3630, 0.99999994),
 (3631, 0.99561656),
 (3632, 0.9955983),
 (3628, 0.99470216),
 (3629, 0.98325384),
 (3574, 0.9724397),
 (3575, 0.97168726),
 (3580, 0.97152376),
 (3573, 0.9709163),
 (3576, 0.9708241),
 (3572, 0.9691833),
 (3578, 0.96891415),
 (3579, 0.96885973),
 (3583, 0.9685684),
 (3577, 0.96672416)]

In [35]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['270',
 '1097',
 '318',
 '356',
 '597',
 607,
 1510,
 3229,
 2821,
 1548,
 584,
 2214,
 763,
 138,
 701]

In [28]:
def get_similar_movie(movie_name: int):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    similar_movie = [movie_to_title[i] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [29]:
get_similar_movie(270) 

['Love Affair (1994)',
 'Walk in the Clouds, A (1995)',
 'Up Close and Personal (1996)',
 'Only You (1994)',
 'Mirror Has Two Faces, The (1996)',
 'One Fine Day (1996)',
 'Dying Young (1991)',
 'Practical Magic (1998)',
 'Forces of Nature (1999)',
 'How to Make an American Quilt (1995)']

#### 제가 좋아하는 영화 Love Affair 를 통해 추천 받은 영화는 로멘틱 드라마 장르를 많이 추천했네요.근데 정말 제가 열광적으로 좋아하는 여화가 여섯편이나 추천되었습니다. 굿굿

In [30]:
get_similar_movie(1097)

['E.T. the Extra-Terrestrial (1982)',
 'Close Encounters of the Third Kind (1977)',
 'Back to the Future (1985)',
 'Dark Crystal, The (1982)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Time Bandits (1981)',
 'Big (1988)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Twelve Monkeys (1995)',
 'Color Purple, The (1985)']

#### 또 너무 좋아하는 E.T. 라는 영화를 통해서는 스티븐 스필버그 감독의 작품과 판타지나 공상과학 영화가 추천 되었습니다. 오 꽤나 잘 학습된듯 보입니다.

### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [31]:
user = user_to_idx['babmu']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=10, filter_already_liked_items=True)
movie_recommended

[(476, 0.0013959975),
 (122, 0.0013787132),
 (23, 0.0013512452),
 (2609, 0.001344864),
 (1138, 0.0013351588),
 (2229, 0.001327447),
 (2044, 0.0012863372),
 (887, 0.001272692),
 (2129, 0.0012619665),
 (707, 0.0012599991)]

In [33]:
print(*[[movie_to_title[idx_to_movie[i[0]]],i[0]] for i in movie_recommended], sep = "\n")

['Wrong Trousers, The (1993)', 476]
['Waking Ned Devine (1998)', 122]
["Schindler's List (1993)", 23]
['Great White Hype, The (1996)', 2609]
['Until the End of the World (Bis ans Ende der Welt) (1991)', 1138]
['Thieves (Voleurs, Les) (1996)', 2229]
['Drunken Master (Zui quan) (1979)', 2044]
['Long Walk Home, The (1990)', 887]
['Character (Karakter) (1997)', 2129]
['Sting, The (1973)', 707]


#### 최종적으로 추천된 영화 입니다. 사실 마지막 결과를 보고 조금 실망 했어요. 제가 선호하는 영화장르가 주로 로멘틱 드라마 인데 처음 5개의 정보중 공상과학 영화가 있어서 그런건지 로멘틱 드라마가 많이 보이지 않네요. 사실 다 처음 보는 제목이라 또 영화를 보면 제가 선호하는 것인지도 모르죠 ㅎㅎㅎ

## 총평

### 제가 좋아하는 분야여서 그런지 모처럼 시간 가는줄 모르고 과제를 했네요. 그리고 비교적 친절한 노드 설명 덕분에 코드를 짜느라 머리가 한움큼 빠지는 일이 없어서 너무 감사한 과제였습니다.. 하지만 제가 코드를 이해하고 이리저리 변경할수 있는 실력이 되야겠죠? 아직도 갈길이 멉니다. 